In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

import pymc3 as pm
import theano
import theano.tensor as tt
import arviz as az

In [34]:
#def generate_data(alpha, beta, n=100, p_r=[.4, .6]):
def generate_data(alpha, beta, n=100, p_r={'high_var': [.95, .05], 'low_var': [.5,.5]}):
    
    actions = np.zeros(n, dtype=np.int)
    rewards = np.zeros(n, dtype=np.int)
    #Qs = np.zeros((n, 2))
    #Stores the expected value for each machine in each trial
    Qs = np.zeros((n, 4))

    # Initialize Q table
    Q = np.array([.5, .5])
    for i in range(n):
        # Apply the Softmax transformation
        exp_Q = np.exp(beta*Q)
        prob_a = exp_Q / np.sum(exp_Q)

        # Simulate choice and reward
        a = np.random.choice([0, 1], p=prob_a)
        r = np.random.rand() < p_r[a]

        # Update Q table
        Q[a] = Q[a] + alpha * (r - Q[a])

        # Store values
        actions[i] = a
        rewards[i] = r
        Qs[i] = Q.copy()

    return actions, rewards, Qs

0

Task 1: Change data generation function to emulate machine game (MG) behavior.
Instead of right and left action for one machine, MG has 4 machines with r(pass)=0 but r(play) can be [-5,+495], [+5,-495], [+10, -100] or [-10, +100] with probs [.95, .05] or [.5, .5]

In [39]:
np.zeros((10, 2))

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])